This notebook was developed using Google Colab with an A100 40GB GPU. The scripts rely on specific file paths to load models and LoRA adapters from Google Drive.

To run these experiments, you will need to update the file paths in the relevant cells to match the locations of the models and adapters on your own system. For successful generation, I recommend a GPU with at least 15GB of VRAM.

# Installs, Imports, Model Loading Function

In [ ]:
# Drive Mount
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Packages
!pip install transformers peft bitsandbytes accelerate -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 125.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 100.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 63.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 42.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 103.8 MB/s eta 0:00:00


In [ ]:
!pip install cltk -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 697.2/697.2 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 106.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.7/193.7 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 90.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 92.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.4/38.4 MB 61.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 65.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 114.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 920.2/920.2 kB 64.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 47.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10

In [ ]:
# Restart the Colab session at this point and mount the Drive again, otherwise imports below might fail.

In [ ]:
from cltk import NLP
from cltk.data.fetch import FetchCorpus
corpus_downloader = FetchCorpus(language="lat")
corpus_downloader.import_corpus("lat_models_cltk")

INFO:CLTK:Cloning 'lat_models_cltk' from 'https://github.com/cltk/lat_models_cltk.git'


In [ ]:
# Imports
import torch, sys, os, time, re
import numpy as np
from transformers import (
      AutoModelForCausalLM,
      AutoTokenizer,
      BitsAndBytesConfig,
      pipeline
  )
from peft import PeftModel
import json
import torch.nn.functional as F
from collections import defaultdict
import random
from cltk.prosody.lat.hexameter_scanner import HexameterScanner
from cltk.prosody.lat.macronizer import Macronizer

In [ ]:
# Model Loading

# Available Model Paths
MODELS_CONFIG = {
    "gemma-3-12b-pt": {
        "path": "/content/drive/MyDrive/ClassicalLatinPoetryGeneration/FinalModels/gemma-3-12b-pt",
        "adapters": {
            "gemma-large": {
                "path": "/content/drive/MyDrive/ClassicalLatinPoetryGeneration/FinalModels/gemma-ds3-12b-lora",
                "description": "Gemma 12B fine-tuned on Dataset v3 (Many authors)."
            }
        }
    },
    "gemma-3-12b-it": {
        "path": "/content/drive/MyDrive/ClassicalLatinPoetryGeneration/FinalModels/gemma-3-12b-it",
        "adapters": {
            "gemma-modern": {
                "path": "/content/drive/MyDrive/ClassicalLatinPoetryGeneration/FinalModels/gemma-3-12b-it-ds4-lora",
                "description": "Gemma 12B-IT fine-tuned on Dataset v4 (Many authors + Modern Latin Poetry created by GPT-4o)."
            }
        }
    }
}

In [ ]:
# load_model Function
def load_model(model_name: str):

    print(f"Loading Model: {model_name}")

    base_model_path = None
    adapter_path = None
    adapter_info = None
    base_model_key = None

    for base_name, base_config in MODELS_CONFIG.items():
        if model_name in base_config["adapters"]:
            base_model_path = base_config["path"]
            adapter_info = base_config["adapters"][model_name]
            adapter_path = adapter_info["path"]
            print(f"Found adapter under base model: {base_name}")
            break

    print(f"{adapter_info['description']}")

    # Quantization configuration
    if base_model_key == "gemma-3-12b-it":
        bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_use_double_quant=False,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.bfloat16
        )
    else:
        bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.bfloat16
        )
    # Loading the model and the tokenizer
    print(f"{base_model_path}")
    base_model = AutoModelForCausalLM.from_pretrained(
        base_model_path,
        quantization_config=bnb_config,
        torch_dtype=torch.bfloat16,
        device_map="auto"
    )

    tokenizer = AutoTokenizer.from_pretrained(base_model_path)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    # Loading the LoRA adapter
    model = PeftModel.from_pretrained(base_model, adapter_path)

    # Through experience:
    # Llama models should stay as PeftModel, without being merged to infer as intended.
    # Gemma models should be merged (it is done out of this function) to be able to make inference.

    print(f"'{model_name}' loaded.")
    return model, tokenizer

# Generating Latin Poetry

## gemma-modern: Gemma-3-12B instruct version fine-tuned on DatasetV4 (DatasetV3 + Modern Latin Poetry created by GPT-4o)

In [ ]:
# Available Models:
# 1. gemma-large-modern = Gemma 12B instruct version fine-tuned on DatasetV4 (DatasetV3 + Modern Latin Poetry created by GPT-4o).
# 2. gemma-large = Gemma 12B pre-trained version fine-tuned on DatasetV3 (DatasetV2 + more authors).

#Loading the chosen model
model, tokenizer = load_model("gemma-modern")

Loading Model: gemma-modern
Found adapter under base model: gemma-3-12b-it
Gemma 12B-IT fine-tuned on Dataset v4 (Many authors + Modern Latin Poetry created by GPT-4o).
/content/drive/MyDrive/ClassicalLatinPoetryGeneration/FinalModels/gemma-3-12b-it


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/bnb.py:348: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


'gemma-modern' loaded.


In [ ]:
# Gemma model have to be merged to be able to make inference.
model = model.merge_and_unload()

In [ ]:
# Loading the symbolic tools
scanner = HexameterScanner()
macronizer = Macronizer(tagger="tag_ngram_123_backoff")

In [ ]:
# Checking the newline token ID
newline_token_id = tokenizer.encode("\n", add_special_tokens=False)[0]
print(newline_token_id)

107


    poem_start_template="Write a poem in Latin, on a modern topic and in dactylic hexameter style. Just type the poem. Nothing more.",
    continuation_prompt_template="Continue the following Latin poem. Just write the next line of the poem on a modern topic and in dactylic hexameter style. Do not write anything else.\n{poem_text}",
    final_line_prompt_template="Write the last line of this Latin poem, on a modern topic and in dactylic hexameter style. Just type the final line. Nothing more.\n{poem_text}"

In [ ]:
# Generation Function for IT Model:

def generate_poem_line_by_line(
    generation_topic,
    num_lines_to_generate=4,
    max_retries_per_line=50,
    max_tokens_per_line=150,
    temperature=0.8,
    top_p=0.90,
    top_k=50,
    repetition_penalty=1.2,
    poem_start_template="Write a poem in Latin on a modern topic about '{topic}', in dactylic hexameter style. Just type the poem. Nothing more.",
    continuation_prompt_template="Continue the following Latin poem on a modern topic about '{topic}', in dactylic hexameter style. Write only the very next line of the poem.\n\nPoem so far:\n{poem_text}",
    final_line_prompt_template="Write a final line for the following Latin poem on a modern topic about '{topic}', in dactylic hexameter style. Write only the final line.\n\nPoem so far:\n{poem_text}"
):

    print("Initializing poem generation with an instruct model...")

    validated_lines = [] # Start with an empty list
    total_attempts = 0

    # The loop
    for i in range(num_lines_to_generate):
        print(f"\n--- Generating line {i + 1}/{num_lines_to_generate} ---")

        line_is_valid = False
        for attempt in range(max_retries_per_line):
            total_attempts += 1 # Increment on every attempt
            print(f"Attempt {attempt + 1}/{max_retries_per_line}...")

            current_poem_so_far = "\n".join(validated_lines)

            # Chooses the correct prompt template based on the line number
            if i == 0:
                prompt_template = poem_start_template
                prompt_content = prompt_template.format(topic=generation_topic)
            elif i == num_lines_to_generate - 1 and num_lines_to_generate > 1: # The very last line
                prompt_template = final_line_prompt_template
                prompt_content = prompt_template.format(topic=generation_topic, poem_text=current_poem_so_far)
            else: # Any line in the middle
                prompt_template = continuation_prompt_template
                prompt_content = prompt_template.format(topic=generation_topic, poem_text=current_poem_so_far)

            chat_prompt = [{"role": "user", "content": prompt_content}]

            print("--- Current Prompt (for instruct model) ---")
            print(prompt_content)
            print("-------------------------------------------")

            # Using the chat template to format the input
            input_ids = tokenizer.apply_chat_template(
                chat_prompt,
                add_generation_prompt=True,
                return_tensors="pt"
            ).to(model.device)

            outputs = model.generate(
                input_ids,
                max_new_tokens=max_tokens_per_line,
                do_sample=True,
                temperature=temperature,
                top_k=top_k,
                top_p=top_p,
                repetition_penalty=repetition_penalty,
                eos_token_id=[tokenizer.eos_token_id, newline_token_id]
            )

            newly_generated_ids = outputs[0][input_ids.shape[-1]:]
            generated_line = tokenizer.decode(newly_generated_ids, skip_special_tokens=True).strip()

            print(f"Candidate line: '{generated_line}'")

            # Validation Step
            try:
                if not generated_line.strip():
                    print("❌ Candidate line is empty. Discarding.")
                    continue

                # 1. Clean the line for the macronizer by removing punctuation
                cleaned_for_macronizer = re.sub(r"[^a-zA-Z\s]", "", generated_line)
                print(f"Cleaned line for macronizer: '{cleaned_for_macronizer}'")

                # 2. Macronize the cleaned line
                macronized_line = macronizer.macronize_text(cleaned_for_macronizer)
                print(f"Macronized line for scanner: '{macronized_line}'")

                # 3. Scan the macronized line for validity
                scan_result = scanner.scan(macronized_line)

                if scan_result.valid:
                    print(f"✅ Line is a valid hexameter. Accepting.")
                    validated_lines.append(generated_line)
                    line_is_valid = True
                    break
                else:
                    print(f"❌ Line is not a valid hexameter. Discarding.")
            except Exception as e:
                print(f"❌ An error occurred during processing: {e}. Discarding line.")

        if not line_is_valid:
            print(f"Stopping generation: Failed to generate a valid line after {max_retries_per_line} attempts.")
            break

    print("\n--- Final Poem ---")
    final_poem = "\n".join(validated_lines)
    # Removing numbers, if exist.
    final_poem = re.sub(r'\d+', '', final_poem)
    print(final_poem)
    # Return the poem AND the total number of attempts
    return final_poem, total_attempts

In [ ]:
generated_poem = generate_poem_line_by_line(generation_topic="war")

In [ ]:
# Batch Generator with given topics

TOPICS_LIST = [
    "romantic Love & existentialism",
    "contemporary Experiences & emotions",
    "science & technology",
    "everyday modern life",
    "abstract modern concepts"
]
POEMS_PER_TOPIC = 10  # This is adjustable
LINES_PER_POEM = 4
OUTPUT_DIR = "/content/drive/MyDrive/ClassicalLatinPoetryGeneration/Generated"
OUTPUT_FILENAME = "gemma-modern_contemporary_passages_by_topic2.txt"


# Batch Generator

def run_batch_generation():
    all_poems_by_topic = {topic: [] for topic in TOPICS_LIST}
    total_attempts_list = []
    total_time_list = []
    total_poems_generated = 0

    print(f"\n{'='*20} STARTING BATCH GENERATION {'='*20}")
    print(f"Will generate {POEMS_PER_TOPIC} poems for each of the {len(TOPICS_LIST)} topics.")

    # Outer loop iterates through the list of topics
    for topic in TOPICS_LIST:
        print(f"\n{'='*15} STARTING TOPIC: '{topic}' {'='*15}")
        poems_for_this_topic = []

        # Inner loop generates the desired number of poems for the current topic
        for i in range(POEMS_PER_TOPIC):
            print(f"\n--- Generating Poem {i + 1}/{POEMS_PER_TOPIC} for topic '{topic}' ---")

            start_time = time.time()

            # Calls the main generator function with the current topic
            generated_poem, attempts = generate_poem_line_by_line(
                generation_topic=topic,
                num_lines_to_generate=LINES_PER_POEM
            )

            end_time = time.time()

            # Stores results only if the poem is complete
            if generated_poem and len(generated_poem.split('\n')) == LINES_PER_POEM:
                poems_for_this_topic.append(generated_poem)
                total_attempts_list.append(attempts)
                total_time_list.append(end_time - start_time)
                total_poems_generated += 1
            else:
                print(f"Poem {i + 1} for topic '{topic}' was not fully generated and will be excluded.")

        all_poems_by_topic[topic] = poems_for_this_topic

    print(f"\n{'='*20} BATCH GENERATION COMPLETE {'='*20}")
    print(f"Successfully generated a total of {total_poems_generated} complete poems.")

    # Statistical Calculations
    if total_poems_generated == 0:
        print("No poems were successfully generated. Exiting.")
        return

    total_lines_generated = total_poems_generated * LINES_PER_POEM
    total_attempts_overall = sum(total_attempts_list)
    avg_attempts_per_line = total_attempts_overall / total_lines_generated if total_lines_generated > 0 else 0
    avg_time_per_passage = np.mean(total_time_list) if total_time_list else 0

    # --- Format the Output File Content ---
    file_content = f"{LINES_PER_POEM}-line dactylic hexameter passages:\n\n"
    file_content += f"--- Overall Generation Statistics ---\n"
    file_content += f"Total Poems Generated: {total_poems_generated}\n"
    file_content += f"Average attempts to find a valid line: {avg_attempts_per_line:.2f}\n"
    file_content += f"Average time to generate one passage: {avg_time_per_passage:.2f} seconds\n"
    file_content += "-------------------------------------\n\n"

    passage_counter = 1
    for topic, poems in all_poems_by_topic.items():
        if poems:
            file_content += f"--- Topic: {topic} ---\n\n"
            for poem in poems:
                file_content += f"Passage {passage_counter}:\n"
                file_content += poem + "\n\n"
                passage_counter += 1

    # Saving
    try:
        os.makedirs(OUTPUT_DIR, exist_ok=True)
        output_path = os.path.join(OUTPUT_DIR, OUTPUT_FILENAME)

        with open(output_path, "w", encoding="utf-8") as f:
            f.write(file_content)

        print(f"\nSuccessfully saved {total_poems_generated} poems and stats to:\n{output_path}")

    except Exception as e:
        print(f"\n❌ An error occurred while saving the file: {e}")
        print("\n--- Generated Content (displaying here since file save failed) ---")
        print(file_content)

In [ ]:
run_batch_generation()


==================== STARTING BATCH GENERATION ====================
Will generate 10 poems for each of the 5 topics.

=============== STARTING TOPIC: 'romantic Love & existentialism' ===============

--- Generating Poem 1/10 for topic 'romantic Love & existentialism' ---
Initializing poem generation with an instruct model...

--- Generating line 1/4 ---
Attempt 1/50...
--- Current Prompt (for instruct model) ---
Write a poem in Latin on a modern topic about 'romantic Love & existentialism', in dactylic hexameter style. Just type the poem. Nothing more.
-------------------------------------------


INFO:CLTK:Tagger tag_ngram_123_backoff could not tag dubia.


Candidate line: 'Amor et vita, dubia mensura rerum;'
Cleaned line for macronizer: 'Amor et vita dubia mensura rerum'
Macronized line for scanner: 'amor et vītā dubia mēnsūrā rērum'
✅ Line is a valid hexameter. Accepting.

--- Generating line 2/4 ---
Attempt 1/50...
--- Current Prompt (for instruct model) ---
Continue the following Latin poem on a modern topic about 'romantic Love & existentialism', in dactylic hexameter style. Write only the very next line of the poem.

Poem so far:
Amor et vita, dubia mensura rerum;
-------------------------------------------


INFO:CLTK:Multiple matching entries found for quae.
INFO:CLTK:Tagger tag_ngram_123_backoff could not tag adest.
INFO:CLTK:Tagger tag_ngram_123_backoff could not tag carnem.
INFO:CLTK:Tagger tag_ngram_123_backoff could not tag spatiumque.
INFO:CLTK:Tagger tag_ngram_123_backoff could not tag vacant.


Candidate line: 'Quae nihil adest praeter carnem spatiumque vacant.'
Cleaned line for macronizer: 'Quae nihil adest praeter carnem spatiumque vacant'
Macronized line for scanner: 'quae nihil adest praeter carnem spatiumque vacant'
❌ Line is not a valid hexameter. Discarding.
Attempt 2/50...
--- Current Prompt (for instruct model) ---
Continue the following Latin poem on a modern topic about 'romantic Love & existentialism', in dactylic hexameter style. Write only the very next line of the poem.

Poem so far:
Amor et vita, dubia mensura rerum;
-------------------------------------------


...

Candidate line: 'Saepe cogito quod haec mens nostra ludunt—neque vera reperio.'
Cleaned line for macronizer: 'Saepe cogito quod haec mens nostra luduntneque vera reperio'
Macronized line for scanner: 'saepe cogito quod haec mēns nostra luduntneque vēra reperio'
❌ Line is not a valid hexameter. Discarding.
Attempt 2/50...
--- Current Prompt (for instruct model) ---
Write a final line for the following Latin poem on a modern topic about 'abstract modern concepts', in dactylic hexameter style. Write only the final line.

Poem so far:
Nunc, mundus post tempus antiquum et vetustum,
Quidnam est hoc "quantum" vocamus? dicite mihi!
Numquid elementa rerum quae nunc credimus?
-------------------------------------------


INFO:CLTK:Tagger tag_ngram_123_backoff could not tag fugeat.
INFO:CLTK:No matching Morpheus entry found for nihil.
INFO:CLTK:Tagger tag_ngram_123_backoff could not tag cognoscemuslatin.


Candidate line: 'Sic fugeat mens; nam nihil iam cognoscemus.{Latin}'
Cleaned line for macronizer: 'Sic fugeat mens nam nihil iam cognoscemusLatin'
Macronized line for scanner: 'sīc fugeat mēns nam nihil jam cognoscemuslatin'
❌ Line is not a valid hexameter. Discarding.
Attempt 3/50...
--- Current Prompt (for instruct model) ---
Write a final line for the following Latin poem on a modern topic about 'abstract modern concepts', in dactylic hexameter style. Write only the final line.

Poem so far:
Nunc, mundus post tempus antiquum et vetustum,
Quidnam est hoc "quantum" vocamus? dicite mihi!
Numquid elementa rerum quae nunc credimus?
-------------------------------------------


INFO:CLTK:Multiple matching entries found for lucis.
INFO:CLTK:Multiple matching entries found for umbrae.


Candidate line: 'Hic tamen error lucis atque umbrae iacet.'
Cleaned line for macronizer: 'Hic tamen error lucis atque umbrae iacet'
Macronized line for scanner: 'hīc tamen error lūcīs atque umbrae jacet'
✅ Line is a valid hexameter. Accepting.

--- Final Poem ---
Nunc, mundus post tempus antiquum et vetustum,
Quidnam est hoc "quantum" vocamus? dicite mihi!
Numquid elementa rerum quae nunc credimus?
Hic tamen error lucis atque umbrae iacet.

==================== BATCH GENERATION COMPLETE ====================
Successfully generated a total of 50 complete poems.

Successfully saved 50 poems and stats to:
/content/drive/MyDrive/ClassicalLatinPoetryGeneration/Generated/gemma-modern_contemporary_passages_by_topic2.txt


## gemma-large: Gemma-3-12B-pre-trained fine-tuned on DatasetV3 (Many Authors)

In [ ]:
# Available Models:
# 1. gemma-large-modern = Gemma 12B instruct version fine-tuned on DatasetV4 (DatasetV3 + Modern Latin Poetry created by GPT-4o).
# 2. gemma-large = Gemma 12B pre-trained version fine-tuned on DatasetV3 (DatasetV2 + more authors).

#Loading the chosen model
model, tokenizer = load_model("gemma-large")

In [ ]:
# Gemma model have to be merged to be able to make inference.
model = model.merge_and_unload()

In [ ]:
# Loading the symbolic tools
scanner = HexameterScanner()
macronizer = Macronizer(tagger="tag_ngram_123_backoff")

In [ ]:
# Checking the newline token ID
newline_token_id = tokenizer.encode("\n", add_special_tokens=False)[0]
print(newline_token_id)

107


In [ ]:
# Generation Function for the Pretrained Model:


def generate_poem_line_by_line(
    initial_prompt,
    num_lines_to_generate=2,
    max_retries_per_line=50,
    max_tokens_per_line=70,
    temperature=0.6,
    top_k=50,
    top_p=0.95,
    repetition_penalty=1.2
):

    print("Initializing poem generation with a pretrained model...")

    full_poem_text = initial_prompt.strip() + "\n"
    validated_lines = []
    total_attempts = 0

    for i in range(num_lines_to_generate):
        print(f"\n--- Generating line {i + 1}/{num_lines_to_generate} ---")

        line_is_valid = False
        for attempt in range(max_retries_per_line):
            total_attempts += 1 # Increment on every attempt
            print(f"Attempt {attempt + 1}/{max_retries_per_line}...")

            print("--- Current Prompt ---")
            print(full_poem_text.strip())
            print("----------------------")

            input_ids = tokenizer(
                full_poem_text,
                return_tensors="pt"
            ).input_ids.to(model.device)

            outputs = model.generate(
                input_ids,
                max_new_tokens=max_tokens_per_line,
                do_sample=True,
                temperature=temperature,
                top_k=top_k,
                top_p=top_p,
                repetition_penalty=repetition_penalty,
                eos_token_id=[tokenizer.eos_token_id, newline_token_id]
            )

            newly_generated_ids = outputs[0][input_ids.shape[-1]:]
            generated_line = tokenizer.decode(newly_generated_ids, skip_special_tokens=True).strip()

            print(f"Candidate line: '{generated_line}'")

            # Validation Step
            try:
                if not generated_line.strip():
                    print("❌ Candidate line is empty. Discarding.")
                    continue

                # 1. Clean the line for the macronizer by removing punctuation
                cleaned_for_macronizer = re.sub(r"[^a-zA-Z\s]", "", generated_line)
                print(f"Cleaned line for macronizer: '{cleaned_for_macronizer}'")

                # 2. Macronize the cleaned line
                macronized_line = macronizer.macronize_text(cleaned_for_macronizer)
                print(f"Macronized line for scanner: '{macronized_line}'")

                # 3. Scan the macronized line for validity
                scan_result = scanner.scan(macronized_line)

                if scan_result.valid:
                    print(f"✅ Line is a valid hexameter. Accepting.")
                    validated_lines.append(generated_line)
                    full_poem_text = initial_prompt.strip() + "\n" + "\n".join(validated_lines) + "\n"
                    line_is_valid = True
                    break
                else:
                    print(f"❌ Line is not a valid hexameter. Discarding.")
            except Exception as e:
                print(f"❌ An error occurred during processing: {e}. Discarding line.")

        if not line_is_valid:
            print(f"Stopping generation: Failed to generate a valid line after {max_retries_per_line} attempts.")
            # If a line fails, we still return what we have so far
            break

    print("\n--- Final Poem ---")
    final_poem = "\n".join(validated_lines)
    # Removing numbers, if exist.
    final_poem = re.sub(r'\d+', '', final_poem)
    print(final_poem)
    # Return the poem and the total number of attempts
    return final_poem, total_attempts

In [ ]:
# This is the special token used during training and a newline character
initial_prompt = "<unused0>\n"

In [ ]:
generated_poem = generate_poem_line_by_line(initial_prompt)

Initializing poem generation with a pretrained model...

--- Generating line 1/2 ---
Attempt 1/50...
--- Current Prompt ---
<unused0>
----------------------


INFO:CLTK:Multiple matching entries found for iuppiter.
INFO:CLTK:Tagger tag_ngram_123_backoff could not tag sedetque.


Candidate line: 'Iuppiter et Iuno, sedetque omnipotens Aiax.'
Cleaned line for macronizer: 'Iuppiter et Iuno sedetque omnipotens Aiax'
Macronized line for scanner: 'juppiter et jūnō sedetque omnipotēns ajāx'
❌ Line is not a valid hexameter. Discarding.
Attempt 2/50...
--- Current Prompt ---
<unused0>
----------------------


INFO:CLTK:No matching Morpheus entry found for primum.
INFO:CLTK:Multiple matching entries found for tenebris.
INFO:CLTK:Tagger tag_ngram_123_backoff could not tag expleta.
ERROR:cltk.prosody.lat.metrical_validator:err at: ------------UU-U, pop from empty list


Candidate line: 'At ubi primum noctis tenebris expleta recessit,'
Cleaned line for macronizer: 'At ubi primum noctis tenebris expleta recessit'
Macronized line for scanner: 'at ubi prīmum noctis tenebrīs expleta recessit'
❌ Line is not a valid hexameter. Discarding.
Attempt 3/50...
--- Current Prompt ---
<unused0>
----------------------


...

Candidate line: ''Non enim erat ille, quem conspexit in aere.'
Cleaned line for macronizer: 'Non enim erat ille quem conspexit in aere'
Macronized line for scanner: 'nōn enim erat ille quem cōnspexit in aere'
❌ Line is not a valid hexameter. Discarding.
Attempt 10/50...
--- Current Prompt ---
<unused0>
----------------------


INFO:CLTK:No matching Morpheus entry found for itaque.
INFO:CLTK:Multiple matching entries found for est.
INFO:CLTK:Multiple matching entries found for qui.
INFO:CLTK:Tagger tag_ngram_123_backoff could not tag exspectet.


Candidate line: ''Itaque, si quisquam est qui te non exspectet.'
Cleaned line for macronizer: 'Itaque si quisquam est qui te non exspectet'
Macronized line for scanner: 'itaque sī quisquam est quī tē nōn exspectet'
✅ Line is a valid hexameter. Accepting.

--- Generating line 2/2 ---
Attempt 1/50...
--- Current Prompt ---
<unused0>
'Itaque, si quisquam est qui te non exspectet.
----------------------


INFO:CLTK:Multiple matching entries found for quis.
INFO:CLTK:Multiple matching entries found for amicus.
INFO:CLTK:Multiple matching entries found for quis.
INFO:CLTK:Tagger tag_ngram_123_backoff could not tag inimicus.


Candidate line: 'quis tibi nunc sit amicus et quis inimicus?'
Cleaned line for macronizer: 'quis tibi nunc sit amicus et quis inimicus'
Macronized line for scanner: 'quis tibi nunc sit amīcus et quis inimicus'
✅ Line is a valid hexameter. Accepting.

--- Final Poem ---
'Itaque, si quisquam est qui te non exspectet.
quis tibi nunc sit amicus et quis inimicus?


In [ ]:
# BATCH GENERATOR

NUM_POEMS_TO_GENERATE = 50
LINES_PER_POEM = 4
OUTPUT_DIR = "/content/drive/MyDrive/ClassicalLatinPoetryGeneration/Generated"
OUTPUT_FILENAME = "gemma-large-generated_passages.txt"

# Batch Generator

def run_batch_generation():
    all_poems = []
    total_attempts_list = []
    total_time_list = []

    print(f"\n{'='*20} STARTING BATCH GENERATION {'='*20}")
    print(f"Will generate {NUM_POEMS_TO_GENERATE} poems of {LINES_PER_POEM} lines each.")

    for i in range(NUM_POEMS_TO_GENERATE):
        print(f"\n--- Generating Poem {i + 1}/{NUM_POEMS_TO_GENERATE} ---")
        initial_prompt = "<unused0>\n"

        start_time = time.time()

        # Call the generator function
        generated_poem, attempts = generate_poem_line_by_line(
            initial_prompt,
            num_lines_to_generate=LINES_PER_POEM
        )

        end_time = time.time()

        # Store results
        if generated_poem: # Only store if a poem was actually generated
            all_poems.append(generated_poem)
            total_attempts_list.append(attempts)
            total_time_list.append(end_time - start_time)

    print(f"\n{'='*20} BATCH GENERATION COMPLETE {'='*20}")

    # Statistical Calculations
    if not all_poems:
        print("No poems were successfully generated. Exiting.")
        return

    # Avg attempts per valid line
    total_lines_generated = len(all_poems) * LINES_PER_POEM
    total_attempts_overall = sum(total_attempts_list)
    avg_attempts_per_line = total_attempts_overall / total_lines_generated if total_lines_generated > 0 else 0

    # Avg time per passage
    avg_time_per_passage = np.mean(total_time_list)

    # File format
    file_content = "4-line dactylic hexameter passages generated by gemma-large:\n\n"
    file_content += f"--- Generation Statistics ---\n"
    file_content += f"Average attempts to find a valid line: {avg_attempts_per_line:.2f}\n"
    file_content += f"Average time to generate one passage: {avg_time_per_passage:.2f} seconds\n"
    file_content += "---------------------------\n\n"

    for idx, poem in enumerate(all_poems):
        file_content += f"Passage {idx + 1}:\n"
        file_content += poem + "\n\n"

    # Saving
    try:
        os.makedirs(OUTPUT_DIR, exist_ok=True)
        output_path = os.path.join(OUTPUT_DIR, OUTPUT_FILENAME)

        with open(output_path, "w", encoding="utf-8") as f:
            f.write(file_content)

        print(f"\nSuccessfully saved {len(all_poems)} poems and stats to:\n{output_path}")

    except Exception as e:
        print(f"\n❌ An error occurred while saving the file: {e}")
        print("\n--- Generated Content (displaying here since file save failed) ---")
        print(file_content)

In [ ]:
run_batch_generation()


==================== STARTING BATCH GENERATION ====================
Will generate 50 poems of 4 lines each.

--- Generating Poem 1/50 ---
Initializing poem generation with a pretrained model...

--- Generating line 1/4 ---
Attempt 1/50...
--- Current Prompt ---
<unused0>
----------------------
Candidate line: ''At non utinam nobis cum primis arma ferre'
Cleaned line for macronizer: 'At non utinam nobis cum primis arma ferre'
Macronized line for scanner: 'at nōn utinam nōbīs cum prīmīs arma ferre'
❌ Line is not a valid hexameter. Discarding.
Attempt 2/50...
--- Current Prompt ---
<unused0>
----------------------


INFO:CLTK:Tagger tag_ngram_123_backoff could not tag superat.
INFO:CLTK:No matching Morpheus entry found for ut.


Candidate line: 'Iamque adeo superat, ut non possit ire retro:'
Cleaned line for macronizer: 'Iamque adeo superat ut non possit ire retro'
Macronized line for scanner: 'jamque adeō superat ut nōn possit īre retrō'
❌ Line is not a valid hexameter. Discarding.
Attempt 3/50...
--- Current Prompt ---
<unused0>
----------------------


INFO:CLTK:Multiple matching entries found for quod.
INFO:CLTK:Multiple matching entries found for est.


Candidate line: ''Nunc age, quod est in animo: quaeso mihi dicere.''
Cleaned line for macronizer: 'Nunc age quod est in animo quaeso mihi dicere'
Macronized line for scanner: 'nunc age quod est in animō quaesō mihi dīcere'
❌ Line is not a valid hexameter. Discarding.
Attempt 4/50...
--- Current Prompt ---
<unused0>
----------------------


...

Candidate line: 'fuit in aves deus; nunc etiam magno praesidio est:'
Cleaned line for macronizer: 'fuit in aves deus nunc etiam magno praesidio est'
Macronized line for scanner: 'fuit in avēs deus nunc etiam magnō praesidiō est'
✅ Line is a valid hexameter. Accepting.

--- Generating line 4/4 ---
Attempt 1/50...
--- Current Prompt ---
<unused0>
'Aurea qui primus vitam, quaeque ad astra relatus
est cum corpore suo per volitantia membra saevas
fuit in aves deus; nunc etiam magno praesidio est:
----------------------


INFO:CLTK:Multiple matching entries found for quid.
INFO:CLTK:Tagger tag_ngram_123_backoff could not tag monui.
INFO:CLTK:Tagger tag_ngram_123_backoff could not tag ovidius.
INFO:CLTK:Tagger tag_ngram_123_backoff could not tag metamorphosis.
INFO:CLTK:Tagger tag_ngram_123_backoff could not tag and.
INFO:CLTK:Tagger tag_ngram_123_backoff could not tag now.
INFO:CLTK:Tagger tag_ngram_123_backoff could not tag you.
INFO:CLTK:Tagger tag_ngram_123_backoff could not tag see.
INFO:CLTK:Tagger tag_ngram_123_backoff could not tag what.
INFO:CLTK:Tagger tag_ngram_123_backoff could not tag that.
INFO:CLTK:Tagger tag_ngram_123_backoff could not tag makes.
INFO:CLTK:Tagger tag_ngram_123_backoff could not tag the.
INFO:CLTK:Tagger tag_ngram_123_backoff could not tag bird.
INFO:CLTK:Tagger tag_ngram_123_backoff could not tag sing.
INFO:CLTK:Tagger tag_ngram_123_backoff could not tag the.
INFO:CLTK:Tagger tag_ngram_123_backoff could not tag nightingale.
INFO:CLTK:Tagger tag_ngram_123_backoff could not

Candidate line: 'quid si nescis? nam saepe tibi monui.' 'Ovidius Metamorphosis 15.243-7]

[And now you see what it is that makes the bird sing.]


[The nightingale was once Philomela who had been raped by Tereus (brother of Procne), and when'
Cleaned line for macronizer: 'quid si nescis nam saepe tibi monui Ovidius Metamorphosis 

And now you see what it is that makes the bird sing


The nightingale was once Philomela who had been raped by Tereus brother of Procne and when'
Macronized line for scanner: 'quid sī nescīs nam saepe tibi monui ovidius metamorphosis and now you see what it is that makes the bird sing the nightingale was once philomela who had been raped by tereus brother of procne and when'
❌ Line is not a valid hexameter. Discarding.
Attempt 2/50...
--- Current Prompt ---
<unused0>
'Aurea qui primus vitam, quaeque ad astra relatus
est cum corpore suo per volitantia membra saevas
fuit in aves deus; nunc etiam magno praesidio est:
----------------------


INFO:CLTK:Multiple matching entries found for caelum.


Candidate line: 'nam caelum et terras atque ipsa domina rerum'
Cleaned line for macronizer: 'nam caelum et terras atque ipsa domina rerum'
Macronized line for scanner: 'nam caelum et terrās atque ipsa dominā rērum'
✅ Line is a valid hexameter. Accepting.

--- Final Poem ---
'Aurea qui primus vitam, quaeque ad astra relatus
est cum corpore suo per volitantia membra saevas
fuit in aves deus; nunc etiam magno praesidio est:
nam caelum et terras atque ipsa domina rerum

==================== BATCH GENERATION COMPLETE ====================

Successfully saved 50 poems and stats to:
/content/drive/MyDrive/ClassicalLatinPoetryGeneration/Generated/gemma-large-generated_passages.txt
